In [3]:
import numpy as np
import matplotlib.pyplot as plt
from sympy import symbols, sin, tan, asin, exp
import mpl_toolkits.mplot3d as Axes3D

# 设置中文字体显示
plt.rcParams["font.family"] = ["SimHei", "WenQuanYi Micro Hei", "Heiti TC"]
plt.rcParams["axes.unicode_minus"] = False  # 正确显示负号

# 参数设置
N = 15                  # 透射光研究条数
lambda_ = 500           # 波长
n = 10                  # 折射率
h = 25000               # 平板厚度
Ai = 1                  # 入射光振幅
a = np.pi / 4           # 入射光振幅方位角
theta_max = np.pi / 3   # 像方视场角
delta_theta = 0.001     # 精度
Asi = Ai * np.sin(a)    # 入射s波振幅
Api = Ai * np.cos(a)    # 入射p波振幅

# 定义菲涅尔公式和相关函数
def Rs(i, r):
    """菲涅尔反射系数(s波)"""
    return -np.sin(i - r) / np.sin(i + r)

def Rp(i, r):
    """菲涅尔反射系数(p波)"""
    return np.tan(i - r) / np.tan(i + r)

def Ts(i, r):
    """菲涅尔透射系数(s波)"""
    return 2 * np.sin(r) * np.cos(i) / np.sin(i + r)

def Tp(i, r):
    """菲涅尔透射系数(p波)"""
    return 2 * np.sin(r) * np.cos(i) / (np.sin(i + r) * np.cos(i - r))

def P(rs, rp):
    """反射比"""
    return (rs * np.sin(a))**2 + (rp * np.cos(a))** 2

# 生成角度数组
i1 = np.arange(-theta_max, theta_max + delta_theta, delta_theta)
r1 = np.arcsin(np.sin(i1) / n)  # 折射角计算
delta = 4 * np.pi / lambda_ * n * h * np.cos(r1)  # 相位差

# 分s和p波计算透射光强 - 修复复数类型问题
As = Asi * Ts(i1, r1) * Ts(r1, i1)
Ap = Api * Tp(i1, r1) * Tp(r1, i1)

# 初始化为复数类型数组，解决类型不匹配问题
Ast = As.astype(complex)
Apt = Ap.astype(complex)

for i in range(1, N):
    # 计算反射和相位项
    rs_term = Rs(r1, i1)**2 * np.exp(1j * delta)
    rp_term = Rp(r1, i1)** 2 * np.exp(1j * delta)
    
    # 更新振幅
    As = As * rs_term
    Ap = Ap * rp_term
    
    # 累加各级振幅
    Ast += As
    Apt += Ap

# 计算总透射光强
It_1 = np.abs(Ast)**2 + np.abs(Apt)** 2

# 各级透射光振幅方向分析
ii = np.pi / 4  # 固定入射角45度
i_sym = symbols('i')  # 定义符号变量
r_sym = asin(sin(i_sym) / n)  # 符号计算折射角

# 计算s波和p波振幅
As_sym = Asi * Ts(ii, r_sym) * Ts(r_sym, ii)
Ap_sym = Api * Tp(ii, r_sym) * Tp(r_sym, ii)

# 计算各级振幅
sols = np.zeros((2, N), dtype=object)
solp = np.zeros((2, N), dtype=object)
sols[0, 0] = 0
solp[0, 0] = 0
sols[1, 0] = As_sym
solp[1, 0] = Ap_sym

for i in range(1, N):
    power = 2 * i
    sols[1, i] = As_sym * (Rs(ii, r_sym) **power)
    solp[1, i] = Ap_sym * (Rp(ii, r_sym)** power)

# 教材中的公式计算
rs_vals = Rs(i1, r1)
rp_vals = Rp(i1, r1)
p_vals = P(rs_vals, rp_vals)
It_2 = (1 - p_vals)**2 / ((1 - p_vals)** 2 + 4 * p_vals * np.sin(delta / 2)**2) * (Ai**2)

# 绘图
# 图1：3D表面图
fig1 = plt.figure(1, figsize=(10, 8))
ax1 = fig1.add_subplot(111, projection='3d')
X, Y = np.meshgrid(i1, i1)
I = np.tile(It_1, (len(i1), 1))
surf = ax1.plot_surface(X, Y, I, cmap='hot', shading='interp')
ax1.set_aspect('equal')
ax1.view_init(90, 90)
fig1.colorbar(surf, shrink=0.5, aspect=5)
plt.title('透射光强分布')

# 图2：两种计算结果对比和振幅方向分析
fig2 = plt.figure(2, figsize=(16, 8))

# 子图1：两种计算结果对比
ax2_1 = fig2.add_subplot(121)
ax2_1.plot(i1, It_1, 'b', label='分sp波', linewidth=1.5)
ax2_1.plot(i1, It_2, 'r', label='教材', linewidth=1.5)
ax2_1.legend()
ax2_1.set_xlabel('干涉角θ', fontsize=14)
ax2_1.set_ylabel('光强I', fontsize=14)
ax2_1.set_title('两种计算结果对比', fontsize=16)
ax2_1.grid(True)

# 子图2：各级透射光总振幅方向
ax2_2 = fig2.add_subplot(122)
# 代入具体角度值计算并绘图
angle_values = np.linspace(0, np.pi/2, 100)
for i in range(1, 5):  # 绘制前4级
    s_vals = np.array([sols[1, i].subs(i_sym, angle).evalf() for angle in angle_values], dtype=float)
    p_vals = np.array([solp[1, i].subs(i_sym, angle).evalf() for angle in angle_values], dtype=float)
    ax2_2.plot(s_vals, p_vals, linewidth=1.5, label=f'{i}级')

ax2_2.legend()
ax2_2.set_xlabel('s波振幅', fontsize=14)
ax2_2.set_ylabel('p波振幅', fontsize=14)
ax2_2.set_title('i=45°时各级透射光总振幅方向分析', fontsize=16)
ax2_2.set_aspect('equal')
ax2_2.grid(True)

plt.tight_layout()
plt.show()
    

TypeError: loop of ufunc does not support argument 0 of type asin which has no callable sin method